In [1]:
import os
import json
import unicodedata
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec

In [2]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nazimaniltepe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
json_data = []
for path, dirs, files in os.walk(f"./TrainingSet"):
    for file in files:
        if file.endswith(".json"):
            f = open(os.path.join(path, file), encoding="utf-8-sig")
            json_data.append(json.load(f))
            json_data[-1]["FileName"] = file
            f.close()

In [4]:
sw_file = open('./stopwords2.txt')
stop_words = sw_file.read()
stop_words = stop_words.split("\n")
sw_file.close()

In [5]:
for data_obj in json_data:
    data_obj["Özet"] = re.sub(r"[-\w\.]+@([-\w]+\.)+[-\w]{2,4}", "", data_obj["Özet"])
    data_obj["Özet"] = re.sub(r"https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b(?:[-a-zA-Z0-9()@:%_\+.~#?&\/=]*)", "", data_obj["Özet"])
    data_obj["Özet"] = re.sub(r"<([a-z]+)(?![^>]*\/>)[^>]*>", "", data_obj["Özet"])

In [24]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

In [25]:
words = []
words_repr = []
for data_obj in json_data:
    vec = tokenizer(data_obj["Özet"])
    words.append(vec)
    words_repr.append((data_obj["FileName"], len(vec), vec))

Token indices sequence length is longer than the specified maximum sequence length for this model (19024 > 512). Running this sequence through the model will result in indexing errors


In [11]:
words = []
words_repr = []
for data_obj in json_data:
    vec = []
    for i in sent_tokenize(data_obj["Özet"], language="turkish"):
        for j in word_tokenize(i, language="turkish"):
            vec.append(j.lower())
    words.append(vec)
    words_repr.append((data_obj["FileName"], len(vec), vec))

In [22]:
type(words[0][0][0])

str

In [12]:
model1 = gensim.models.Word2Vec(words, min_count = 1, vector_size = 100, window = 5)

In [15]:
model1.wv['anonim']

array([-1.03946626e+00,  9.95541811e-01,  4.26392764e-01, -1.50332658e-03,
        2.69846201e-01, -1.19535792e+00,  9.16952252e-01,  1.57877541e+00,
       -5.30536592e-01, -6.03813589e-01, -6.53959870e-01, -4.50643957e-01,
       -4.79648292e-01,  7.38062441e-01,  1.07448649e+00, -6.57557964e-01,
        6.93104208e-01,  1.64789140e-01, -5.74939251e-01, -6.57999396e-01,
       -4.83407974e-02, -7.19854310e-02,  8.23006749e-01, -2.95457333e-01,
       -1.46126747e-01, -1.76016420e-01, -2.06016660e-01,  1.69496641e-01,
       -1.01063228e+00, -1.14812590e-01, -2.33470827e-01,  7.16659904e-01,
       -7.52529874e-03, -5.22627890e-01, -4.26430792e-01,  1.16897011e+00,
       -1.39112338e-01, -6.16025686e-01, -4.89752084e-01, -8.35157931e-01,
       -1.36250541e-01, -4.57478434e-01, -1.20815836e-01,  1.61586285e-01,
        8.90136480e-01,  1.86616942e-01, -8.47792998e-02,  2.55777210e-01,
        1.82813630e-02,  8.33013117e-01,  2.72913694e-01, -2.31195405e-01,
        1.97726876e-01, -

In [8]:
words = [(data_obj["FileName"], clean_list(data_obj["Özet"].split(" "))) for data_obj in json_data]
words = [(w[0], len(w[1]), w[1]) for w in words]
w_flat = [(w, wvec[0]) for wvec in words for w in wvec[2]]

In [9]:
keywords = [(data_obj["FileName"], clean_list(data_obj["Anahtar Kelimeler"], True)) for data_obj in json_data]
keywords = [(kw[0], len(kw[1]), kw[1]) for kw in keywords]
kw_flat = [(kw, kwvec[0]) for kwvec in keywords for kw in kwvec[2]]

In [10]:
def unique_chars_and_words(list_flat):
    list_unique = []
    list_dict = {}
    list_chars = []
    list_charcodes = []
    for w in list_flat:
        if w[0] not in list_unique:
            list_unique.append(w[0])
            list_dict[w[0]] = [w[1]]
        else:
            list_dict[w[0]].append(w[1])
        for c in w[0]:
            if c not in list_chars:
                list_chars.append(c)
                list_charcodes.append((ord(c), w[0]))
    list_chars = [[list_chars[i], list_charcodes[i][0], list_charcodes[i][1]] for i, c in enumerate(list_chars)]
    list_unique = [[list_unique[i], len(list_dict[list_unique[i]]), list_dict[list_unique[i]]] for i,_ in enumerate(list_unique)]
    return (list_chars, list_unique)

In [11]:
kw_chars, kw_unique = unique_chars_and_words(kw_flat)
w_chars, w_unique = unique_chars_and_words(w_flat)